In [7]:
from keras import applications, backend, Model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

hx = hy = 150
if backend.image_data_format() == 'channels_first':
    input_shape = (3, hx, hy)
else:
    input_shape = (hx, hy, 3)

# path to the model weights files trained with bottle-neck
top_model_weights_path = 'bottleneck_fc_model.h5'

# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train_small'
validation_data_dir = 'data/val_small'
nb_train_samples = 2000
nb_validation_samples = 200
epochs = 50
batch_size = 25

# build the VGG16 network into model
model = applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# define top model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# load pre-trained top model weights
top_model.load_weights(top_model_weights_path)

# add top model on top of vgg model
# model.add(top_model)
model = Model(inputs=model.input, outputs=top_model(model.output))

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# Keep very slow learning rate to avoid deterioriation of learned parameters
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])


In [8]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [ ]:
# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., validation_steps=200, epochs=50, steps_per_epoch=80)`
  import sys


Epoch 1/50
80/80 [==============================] - 1783s 22s/step - loss: 0.6765 - acc: 0.9305 - val_loss: 1.0941 - val_acc: 0.9000
Epoch 2/50
79/80 [============================>.] - ETA: 6s - loss: 0.6411 - acc: 0.9332 